In [ ]:
!pip install langchain-xai langchain python-dotenv

---

#### 1. Zero-Shot Prompting

- Description: Directly asking the model to perform a task without providing examples. The prompt relies on the model’s pre-trained knowledge to generate a response.

- Use Case: Quick tasks where the model is expected to understand the instruction without further context.

- Explanation: The prompt directly asks for the answer without examples, relying on the model’s knowledge.

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_xai import ChatXAI
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()
api_key = os.getenv("XAI_API_KEY")

# Initialize the language model
llm = ChatXAI(model_name="grok-3", xai_api_key=api_key)

# Define the zero-shot prompt
prompt = PromptTemplate(
    input_variables=["question"],
    template="Answer the following question: {question}"
)

# Create a chain using RunnableSequence
chain = prompt | llm

# Run the chain
question = "What is the capital of France?"
response = chain.invoke({"question": question})

# Handle the response (assuming ChatXAI returns a ChatMessage-like object)
print(f"Prompt: {question}")
print(f"Response: {response.content if hasattr(response, "content") else response}")


---

#### 2. Few-Shot Prompting

- Description: Providing a few examples within the prompt to guide the model’s response format or style. This helps the model understand the task better.

- Use Case: Tasks requiring specific formats or styles, like classification or translation.

- Explanation: The prompt includes examples to show the model how to classify sentiment, guiding it to produce a consistent response.

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_xai import ChatXAI
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()
api_key = os.getenv("XAI_API_KEY")

# Initialize the language model
llm = ChatXAI(model_name="grok-3", xai_api_key=api_key)

# Define the few-shot prompt with examples
prompt = PromptTemplate(
    input_variables=["text"],
    template="Classify the sentiment of the following text as positive, negative, or neutral:\n\n"
             "Example 1: 'I love this product!' -> Positive\n"
             "Example 2: 'This is disappointing.' -> Negative\n"
             "Example 3: 'The item is okay.' -> Neutral\n\n"
             "Text: {text}\nSentiment:"
)

chain = prompt | llm

text = "I'm really impressed with the service!"
response = chain.invoke({"text": text})
print(f"Prompt: {text}")
print(response.content if hasattr(response, "content") else response)

---

#### 3. Chain-of-Thought (CoT) Prompting

- Description: Encouraging the model to reason step-by-step to arrive at an answer, often improving performance on complex tasks like math or logic.

- Use Case: Arithmetic, logical reasoning, or multi-step problem-solving.

- Explanation: The prompt explicitly asks for step-by-step reasoning, which helps the model produce a clear and accurate solution.

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_xai import ChatXAI
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()
api_key = os.getenv("XAI_API_KEY")

# Initialize the language model
llm = ChatXAI(model_name="grok-3", xai_api_key=api_key)

# Define the CoT prompt
prompt = PromptTemplate(
    input_variables=["question"],
    template="Solve the following problem step-by-step:\n{question}\n\nProvide your reasoning and final answer."
)

chain = prompt | llm

question = "What is 15 × 8?"
response = chain.invoke({"question": question})
print(f"Prompt: {question}")
print(response.content if hasattr(response, "content") else response)

---

#### 4. Self-Consistency Prompting

- Description: Generating multiple responses to the same prompt and selecting the most consistent answer (e.g., via majority voting). This reduces errors from variability.

- Use Case: Tasks where the model might produce inconsistent answers, like math or ambiguous questions.

- Explanation: The example generates five responses and uses Counter to select the most frequent one. The temperature=0.7 encourages slight variations in responses.

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_xai import ChatXAI
from dotenv import load_dotenv
from collections import Counter
import os

# Load environment variables
load_dotenv()
api_key = os.getenv("XAI_API_KEY")

# Initialize the language model
llm = ChatXAI(model_name="grok-3", xai_api_key=api_key, temperature=0.7)

# Define the prompt
prompt = PromptTemplate(
    input_variables=["question"],
    template="Solve the following problem: {question}"
)

# Create a chain using RunnableSequence
chain = prompt | llm

# Generate multiple responses
question = "What is 15 × 8?"
responses = [chain.invoke({"question": question}) for _ in range(5)]

# Extract content from responses (handle string or ChatMessage-like objects)
response_contents = [
    resp.content if hasattr(resp, "content") else str(resp) for resp in responses
]

# Normalize responses for majority voting (e.g., extract the number "120")
normalized_responses = []
for content in response_contents:
    # Extract the numeric answer (assuming responses contain "120" or similar)
    # Adjust this logic based on ChatXAI's output format
    import re
    numbers = re.findall(r'\b\d+\b', content)
    normalized_responses.append(numbers[0] if numbers else content.strip())

# Aggregate results using majority voting
answer_counts = Counter(normalized_responses)
final_answer = answer_counts.most_common(1)[0][0]

# Print responses and final answer
print(f"Prompt: {question}")
print(f"Responses: {response_contents}")
print(f"Normalized Responses: {normalized_responses}")
print(f"Final Answer: {final_answer}")

---

#### 5. Role-Based Prompting

- Description: Assigning a specific role or persona to the model (e.g., “You are a teacher”) to tailor the tone or perspective of the response.

- Use Case: Responses requiring a specific tone, expertise, or style, like explaining concepts to a beginner.

- Explanation: The prompt assigns the model a teacher role, ensuring a simple and engaging explanation tailored for students.

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_xai import ChatXAI
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()
api_key = os.getenv("XAI_API_KEY")

# Initialize the language model
llm = ChatXAI(model_name="grok-3", xai_api_key=api_key)

# Define the role-based prompt
prompt = PromptTemplate(
    input_variables=["topic"],
    template="You are a high school science teacher. Explain {topic} in a simple and engaging way for students."
)

chain = prompt | llm

topic = "photosynthesis"
response = chain.invoke({"topic": topic})
print(f"Prompt: {topic}")
print(f"Response: {response.content if hasattr(response, "content") else response}")

---

#### 6. Prompt Chaining

- Description: Breaking a complex task into multiple sequential prompts, where the output of one prompt feeds into the next. This is ideal for multi-step tasks.

- Use Case: Tasks requiring multiple stages, like summarizing then analyzing a text.

- Explanation: The first chain summarizes the text, and the second chain analyzes the sentiment of the summary, demonstrating how outputs can feed into subsequent prompts.

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_xai import ChatXAI
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()
api_key = os.getenv("XAI_API_KEY")

# Initialize the language model
llm = ChatXAI(model_name="grok-3", xai_api_key=api_key)

# First prompt: Summarize text
prompt1 = PromptTemplate(
    input_variables=["text"],
    template="Summarize the following text in one sentence: {text}"
)
chain1 = prompt1 | llm

# Second prompt: Analyze sentiment
prompt2 = PromptTemplate(
    input_variables=["summary"],
    template="What is the sentiment of this summary? {summary}"
)
chain2 = prompt2 | llm

# Combine into a sequential chain
def create_sequential_chain():
    def process_summary(summary):
        # Handle ChatXAI output (ChatMessage-like or string)
        summary_text = summary.content if hasattr(summary, "content") else str(summary)
        return {"summary": summary_text}

    # Chain: summarize text, process output, then analyze sentiment
    overall_chain = chain1 | process_summary | chain2
    return overall_chain

overall_chain = create_sequential_chain()

# Run the chain
text = "The new park in town is beautiful, with vibrant flowers and clean pathways, but the parking is limited."
response = overall_chain.invoke({"text": text})

# Extract final response
response_content = response.content if hasattr(response, "content") else str(response)

# Print results with verbose-like output
print(f"Input Text: {text}")
print(f"Final Response (Sentiment): {response_content}")

---

#### 7. Instruction-Based Prompting

- Description: Providing explicit instructions within the prompt to guide the model’s behavior, such as specifying format, tone, or constraints.

- Use Case: Tasks requiring structured output, like JSON or a specific tone.

- Explanation: The prompt specifies the format (three bullet points) and tone (formal), ensuring the model adheres to these constraints.

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_xai import ChatXAI
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()
api_key = os.getenv("XAI_API_KEY")

# Initialize the language model
llm = ChatXAI(model_name="grok-3", xai_api_key=api_key)

# Define the instruction-based prompt
prompt = PromptTemplate(
    input_variables=["product"],
    template="Describe the product in exactly three bullet points, using a formal tone.\nProduct: {product}"
)

chain = prompt | llm

product = "wireless earbuds"
response = chain.invoke({"product": product})
print(f"Prompt: {product}")
print(f"Response: {response.content if hasattr(response, "content") else response}")

---

#### 8. Contextual Prompting

- Description: Including relevant context or background information in the prompt to improve the model’s understanding and response accuracy.

- Use Case: Tasks requiring domain-specific knowledge or context, like answering questions about a specific document.

- Explanation: The prompt provides specific context about the Eiffel Tower, enabling the model to give a precise answer.

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_xai import ChatXAI
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()
api_key = os.getenv("XAI_API_KEY")

# Initialize the language model
llm = ChatXAI(model_name="grok-3", xai_api_key=api_key)

# Define the contextual prompt
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="Given the following context:\n{context}\n\nAnswer the question: {question}"
)

chain = prompt | llm

context = "The Eiffel Tower, located in Paris, France, was completed in 1889 and stands 324 meters tall."
question = "What is the height of the Eiffel Tower?"
response = chain.invoke({"context": context, "question": question})
print(f"Context: {context}")
print(f"Question: {question}")
print(f"Response: {response.content if hasattr(response, "content") else response}")